# Create dataset


## Quarterly hours for Macrozone Nord

We start considering the Macrozone NORD

In [1]:
import sys
print(sys.executable)

c:\Users\samueledelia\.conda\envs\myenv\python.exe


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_nord = qh[qh['MACROZONA'] == 'NORD']
qh_nord = qh_nord.drop(columns = ['MACROZONA'])

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [3]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_ENERGY_BALANCE" 
power = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power = power.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
power = power[['ENERGY_BALANCE_GWh', 'ORAINI', 'ENERGY_SOURCE']]
# Convert ORAINI to datetime using the correct method
power['ORAINI'] = pd.to_datetime(power['ORAINI'], format='%Y%m%d%H%M')
power.set_index('ORAINI', inplace=True)
thermo = utils.filter_and_rename_energy_source(power, 'Thermal', 'THERMO_GWH')
wind = utils.filter_and_rename_energy_source(power, 'Wind', 'WIND_GWH')
solar = utils.filter_and_rename_energy_source(power, 'Photovoltaic', 'SOLAR_GWH')
hydro = utils.filter_and_rename_energy_source(power, 'Hydro', 'HYDRO_GWH')
geothermal = utils.filter_and_rename_energy_source(power, 'Geothermal', 'GEO_GWH')
forex = utils.filter_and_rename_energy_source(power, 'Net Foreign Exchange', 'FOREX_GWH')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [4]:
# Resample the data to 15-minute intervals and divide by 4
wind = wind.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
wind['WIND_GWH'] = wind['WIND_GWH'] / 4
wind['WIND_MWH'] = wind['WIND_GWH'] * 1000
# Drop the original hourly GWh column
wind = wind.drop(columns=['WIND_GWH'])

wind

,WIND_MWH
ORAINI,
2019-01-01 00:00:00,1275.0
2019-01-01 00:15:00,1275.0
2019-01-01 00:30:00,1275.0
2019-01-01 00:45:00,1275.0
2019-01-01 01:00:00,1257.5
...,...
2024-11-25 22:00:00,400.0
2024-11-25 22:15:00,400.0
2024-11-25 22:30:00,400.0


In [5]:
# Resample the data to 15-minute intervals and divide by 4
solar = solar.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
solar['SOLAR_GWH'] = solar['SOLAR_GWH'] / 4
solar['SOLAR_MWH'] = solar['SOLAR_GWH'] * 1000
# Drop the original hourly GWh column
solar = solar.drop(columns=['SOLAR_GWH'])

solar

,SOLAR_MWH
ORAINI,
2019-01-01 00:00:00,0.0
2019-01-01 00:15:00,0.0
2019-01-01 00:30:00,0.0
2019-01-01 00:45:00,0.0
2019-01-01 01:00:00,0.0
...,...
2024-11-25 22:00:00,0.0
2024-11-25 22:15:00,0.0
2024-11-25 22:30:00,0.0


In [6]:
# Resample the data to 15-minute intervals and divide by 4
thermo = thermo.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
thermo['THERMO_GWH'] = thermo['THERMO_GWH'] / 4
thermo['THERMO_MWH'] = thermo['THERMO_GWH'] * 1000
# Drop the original hourly GWh column
thermo = thermo.drop(columns=['THERMO_GWH'])

thermo

,THERMO_MWH
ORAINI,
2019-01-01 00:00:00,3337.5
2019-01-01 00:15:00,3337.5
2019-01-01 00:30:00,3337.5
2019-01-01 00:45:00,3337.5
2019-01-01 01:00:00,3272.5
...,...
2024-11-25 22:00:00,4937.5
2024-11-25 22:15:00,4937.5
2024-11-25 22:30:00,4937.5


In [7]:
# Resample the data to 15-minute intervals and divide by 4
hydro = hydro.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
hydro['HYDRO_GWH'] = hydro['HYDRO_GWH'] / 4
hydro['HYDRO_MWH'] = hydro['HYDRO_GWH'] * 1000
# Drop the original hourly GWh column
hydro = hydro.drop(columns=['HYDRO_GWH'])

hydro

,HYDRO_MWH
ORAINI,
2019-01-01 00:00:00,515.0
2019-01-01 00:15:00,515.0
2019-01-01 00:30:00,515.0
2019-01-01 00:45:00,515.0
2019-01-01 01:00:00,437.5
...,...
2024-11-25 22:00:00,695.0
2024-11-25 22:15:00,695.0
2024-11-25 22:30:00,695.0


In [8]:
# Resample the data to 15-minute intervals and divide by 4
geothermal = geothermal.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
geothermal['GEO_GWH'] = geothermal['GEO_GWH'] / 4
geothermal['GEO_MWH'] = geothermal['GEO_GWH'] * 1000
# Drop the original hourly GWh column
geothermal = geothermal.drop(columns=['GEO_GWH'])

geothermal

,GEO_MWH
ORAINI,
2019-01-01 00:00:00,167.5
2019-01-01 00:15:00,167.5
2019-01-01 00:30:00,167.5
2019-01-01 00:45:00,167.5
2019-01-01 01:00:00,167.5
...,...
2024-11-25 22:00:00,147.5
2024-11-25 22:15:00,147.5
2024-11-25 22:30:00,147.5


In [9]:
## Resample the data to 15-minute intervals and divide by 4
#forex = forex.resample('15min').ffill()  # Forward fill to maintain hourly data for the quarter-hour 
#forex['FOREX_GWH'] = forex['FOREX_GWH'] / 4
#forex['FOREX_MWH'] = forex['FOREX_GWH'] * 1000
# Drop the original hourly GWh column
#forex = forex.drop(columns=['FOREX_GWH'])

#forex

In [10]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_PHYSICAL_FOREIGN_FLOW" 
physical_forex = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
physical_forex = physical_forex.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
physical_forex = physical_forex[['PHYSICAL_FOREIGN_FLOW_MW', 'ORAINI', 'COUNTRY']]
# Convert ORAINI to datetime using the correct method
physical_forex['ORAINI'] = pd.to_datetime(physical_forex['ORAINI'], format='%Y%m%d%H%M')
physical_forex.set_index('ORAINI', inplace=True)

physical_forex

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,PHYSICAL_FOREIGN_FLOW_MW,COUNTRY
ORAINI,,
2019-01-01 00:00:00,156.0,Austria
2019-01-01 00:00:00,-142.0,Corsica
2019-01-01 00:00:00,-321.0,France
2019-01-01 00:00:00,-313.0,Greece
2019-01-01 00:00:00,-80.0,Malta
...,...,...
2024-11-25 23:00:00,1744.0,France
2024-11-25 23:00:00,494.0,Greece
2024-11-25 23:00:00,-89.0,Malta


In [11]:
austria_physical = utils.process_foreign_exchange(physical_forex, 'Austria')
corsica_physical = utils.process_foreign_exchange(physical_forex, 'Corsica')
france_physical = utils.process_foreign_exchange(physical_forex, 'France')
greece_physical = utils.process_foreign_exchange(physical_forex, 'Greece')
malta_physical = utils.process_foreign_exchange(physical_forex, 'Malta')
slovenia_physical = utils.process_foreign_exchange(physical_forex, 'Slovenia')
switzerland_physical = utils.process_foreign_exchange(physical_forex, 'Switzerland')
montenegno_physical = utils.process_foreign_exchange(physical_forex, 'Montenegno')

In [12]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,1843.786,0.0,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,1845.050,0.0,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,1845.647,0.0,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,983.111,0.0,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,978.775,0.0,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-26 19:00:00,42149.889,43347.924,3576.202,7388.522,23498.975,1055.693,2259.612,2548.437,0.0,None,...,NaN,0.0,3603.000,0.0,0.0,0.0,2786.0,40561.924,42809.889,2024-11-25 13:10:39.207
2024-11-26 20:00:00,39528.253,40688.253,3321.071,6995.963,22051.242,1036.215,2183.551,2485.317,0.0,None,...,NaN,0.0,3579.000,0.0,0.0,243.0,2786.0,37659.314,40188.253,2024-11-25 13:10:39.207
2024-11-26 21:00:00,36475.073,36975.073,3040.707,6432.743,20417.280,960.222,2018.808,2280.772,0.0,None,...,NaN,0.0,3535.838,624.8,0.0,390.0,2808.0,33777.045,36475.073,2024-11-25 13:10:39.207


In [13]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mgp_volumes_nord.rename(columns={"NORD_PURCHASES": "MGP_NORD_PURCHASES", "NORD_SALES": "MGP_NORD_SALES"}, inplace=True)
mgp_volumes_nord = mgp_volumes_nord[~mgp_volumes_nord.index.duplicated(keep='first')]
mgp_volumes_nord = mgp_volumes_nord.resample('15min').ffill()/4


mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
mi1_volumes = mi1_volumes.drop(columns=['MARKET'])
mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mi1_volumes_nord.rename(columns={"NORD_PURCHASES": "MI1_NORD_PURCHASES", "NORD_SALES": "MI1_NORD_SALES"}, inplace=True)
mi1_volumes_nord = mi1_volumes_nord[~mi1_volumes_nord.index.duplicated(keep='first')]
mi1_volumes_nord = mi1_volumes_nord.resample('15min').ffill()/4

In [14]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SCHEDULED_INTERNAL_EXCHANGE"
scheduled_internal = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
scheduled_internal = scheduled_internal.sort_values(by='ORAINI')

# Filter the DataFrame for the relevant columns and rows
scheduled_internal = scheduled_internal[['SCHEDULED_INTERNAL_EXCHANGE_MW', 'ORAINI', 'FLOW_FROM', 'FLOW_TO']]
# Convert ORAINI to datetime using the correct method
scheduled_internal['ORAINI'] = pd.to_datetime(scheduled_internal['ORAINI'], format='%Y%m%d%H%M')
scheduled_internal.set_index('ORAINI', inplace=True)
scheduled_internal = scheduled_internal[(scheduled_internal['FLOW_FROM'] == 'CNOR') & (scheduled_internal['FLOW_TO'] == 'NORD') ]
scheduled_internal = scheduled_internal.drop(columns=['FLOW_FROM','FLOW_TO'])
scheduled_internal_nord = scheduled_internal.resample('15min').ffill()/4

scheduled_internal_nord

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,SCHEDULED_INTERNAL_EXCHANGE_MW
ORAINI,
2019-01-01 00:00:00,158.7875
2019-01-01 00:15:00,158.7875
2019-01-01 00:30:00,158.7875
2019-01-01 00:45:00,158.7875
2019-01-01 01:00:00,235.1475
...,...
2024-11-22 13:00:00,158.5875
2024-11-22 13:15:00,158.5875
2024-11-22 13:30:00,158.5875


In [15]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_TOTAL_LOAD_QH" 
total_load = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
total_load = total_load.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
total_load = total_load[['ZONA', 'TOTAL_LOAD_MW', 'FORECAST_TOTAL_LOAD_MW', 'ORAINI']]
# Convert ORAINI to datetime using the correct method
total_load['ORAINI'] = pd.to_datetime(total_load['ORAINI'], format='%Y%m%d%H%M')
total_load.set_index('ORAINI', inplace=True)
subset_zonas = ['NORD']
mnord_load = utils.filter_load_by_zona(total_load, subset_zonas)
mnord_load = mnord_load.drop(columns=['ZONA'])
mnord_load

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,11776.405,12085.191
2019-01-01 00:15:00,11723.369,12010.132
2019-01-01 00:30:00,11771.133,11984.692
2019-01-01 00:45:00,11740.606,11935.066
2019-01-01 01:00:00,11559.374,11825.838
...,...,...
2024-11-25 22:45:00,19034.956,19603.015
2024-11-25 23:00:00,18526.847,19041.313
2024-11-25 23:15:00,18125.691,18501.986


Total and forecast load ENTSOE

In [16]:
# Load the data
df_load_ensoe_path = r'C:\imbalance_forecast\data\df_load_entsoe.csv'
df_load_ensoe_path = pd.read_csv(df_load_ensoe_path, index_col=0, parse_dates=True)

# Access the relevant columns
mnord_load_entsoe = df_load_ensoe_path[['DAY_AHEAD_TOTAL_LOAD_MW_NORD', 'ACTUAL_TOTAL_LOAD_MW_NORD']]

# Check for duplicate timestamps
duplicates = mnord_load_entsoe.index.duplicated(keep=False)

if duplicates.any():
    # Handle duplicates: average values for duplicate timestamps
    mnord_load_entsoe = mnord_load_entsoe.groupby(mnord_load_entsoe.index).mean()

# Now resample the data
mnord_load_entsoe = mnord_load_entsoe.resample('15min').ffill() / 4

# Display the result
mnord_load_entsoe

,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,
2019-01-01 00:00:00,2777.50,2877.75
2019-01-01 00:15:00,2777.50,2877.75
2019-01-01 00:30:00,2777.50,2877.75
2019-01-01 00:45:00,2777.50,2877.75
2019-01-01 01:00:00,2668.75,2759.75
...,...,...
2024-11-04 11:00:00,NaN,NaN
2024-11-04 11:15:00,NaN,NaN
2024-11-04 11:30:00,NaN,NaN


Lag SBIL

In [17]:
lags = [-1, -4, -96]

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(qh_nord['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged

,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag96
ORAINI,,,
2019-01-01 00:00:00,NaN,NaN,NaN
2019-01-01 00:15:00,-47.312,NaN,NaN
2019-01-01 00:30:00,-30.343,NaN,NaN
2019-01-01 00:45:00,-20.311,NaN,NaN
2019-01-01 01:00:00,-13.934,-47.312,NaN
...,...,...,...
2024-11-26 10:45:00,-90.502,-81.527,-74.785
2024-11-26 11:00:00,-89.338,-84.896,-262.671
2024-11-26 11:15:00,-180.234,-70.471,-270.513


In [18]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [qh_nord, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, austria_physical, france_physical, slovenia_physical, switzerland_physical, scheduled_internal_nord, mgp_volumes_nord, mi1_volumes_nord, mnord_load, mnord_load_entsoe]
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

df_nord = df_nord[df_nord.index <= '2024-03-11']
df_nord = df_nord[df_nord.index >= '2019-01-03']
df_nord.head(35)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag96,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,AUSTRIA_MWQH,...,SWITZERLAND_MWQH,SCHEDULED_INTERNAL_EXCHANGE_MW,MGP_NORD_PURCHASES,MGP_NORD_SALES,MI1_NORD_PURCHASES,MI1_NORD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-03 00:00:00,-1.847,63.465,-156.567,38.978,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,13261.060,13912.337,3194.50,3126.25
2019-01-03 00:15:00,-11.944,-1.847,-63.103,56.932,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,12996.362,13697.814,3194.50,3126.25
2019-01-03 00:30:00,0.336,-11.944,16.441,122.707,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,12869.871,13592.390,3194.50,3126.25
2019-01-03 00:45:00,55.911,0.336,63.465,174.893,1402.5,3242.5,0.0,407.5,167.5,39.25,...,526.00,82.8000,3221.22925,2003.79250,151.94800,106.55450,12624.143,13419.621,3194.50,3126.25
2019-01-03 01:00:00,-17.541,55.911,-1.847,99.216,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12234.627,13132.506,2954.75,2918.50
2019-01-03 01:15:00,-46.081,-17.541,-11.944,104.522,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12286.212,12814.419,2954.75,2918.50
2019-01-03 01:30:00,-21.225,-46.081,0.336,130.053,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12437.186,12880.788,2954.75,2918.50
2019-01-03 01:45:00,29.607,-21.225,55.911,136.617,1360.0,3160.0,0.0,395.0,170.0,40.50,...,398.75,221.8950,3076.62150,1992.72250,129.96975,134.68950,12243.895,12746.228,2954.75,2918.50
2019-01-03 02:00:00,-46.653,29.607,-17.541,106.558,1220.0,3150.0,0.0,405.0,170.0,43.25,...,388.00,298.3425,2990.44050,1977.44700,108.58025,106.28475,12027.606,12552.932,2827.25,2825.25


In [19]:
df_nord.tail(25)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag4,SBIL_MWH_lag96,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,AUSTRIA_MWQH,...,SWITZERLAND_MWQH,SCHEDULED_INTERNAL_EXCHANGE_MW,MGP_NORD_PURCHASES,MGP_NORD_SALES,MI1_NORD_PURCHASES,MI1_NORD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2024-03-10 18:00:00,154.907,-24.060,195.264,-104.128,1440.0,2275.0,2.5,1727.5,155.0,62.50,...,1147.50,46.305,4211.55575,2585.89450,146.23825,203.99550,17952.554,17896.750,4284.75,4339.00
2024-03-10 18:15:00,57.474,154.907,246.132,104.844,1440.0,2275.0,2.5,1727.5,155.0,62.50,...,1147.50,46.305,4211.55575,2585.89450,146.23825,203.99550,18717.886,18559.667,4284.75,4339.00
2024-03-10 18:30:00,-114.593,57.474,65.446,81.467,1440.0,2275.0,2.5,1727.5,155.0,62.50,...,1147.50,46.305,4211.55575,2585.89450,146.23825,203.99550,19556.676,19124.851,4284.75,4339.00
2024-03-10 18:45:00,-207.391,-114.593,-24.060,-90.002,1440.0,2275.0,2.5,1727.5,155.0,62.50,...,1147.50,46.305,4211.55575,2585.89450,146.23825,203.99550,20089.395,19615.101,4284.75,4339.00
2024-03-10 19:00:00,-215.010,-207.391,154.907,-172.590,1325.0,2285.0,0.0,2225.0,155.0,57.50,...,1257.00,46.305,4371.02225,2797.34600,184.84675,319.42550,20399.797,19755.141,4414.00,4587.25
2024-03-10 19:15:00,-211.963,-215.010,57.474,-151.080,1325.0,2285.0,0.0,2225.0,155.0,57.50,...,1257.00,46.305,4371.02225,2797.34600,184.84675,319.42550,20397.057,19928.699,4414.00,4587.25
2024-03-10 19:30:00,-125.881,-211.963,-114.593,-34.305,1325.0,2285.0,0.0,2225.0,155.0,57.50,...,1257.00,46.305,4371.02225,2797.34600,184.84675,319.42550,20394.751,20049.743,4414.00,4587.25
2024-03-10 19:45:00,-36.376,-125.881,-207.391,-73.075,1325.0,2285.0,0.0,2225.0,155.0,57.50,...,1257.00,46.305,4371.02225,2797.34600,184.84675,319.42550,19926.332,19725.274,4414.00,4587.25
2024-03-10 20:00:00,-201.162,-36.376,-215.010,-2.605,1172.5,2297.5,0.0,2045.0,155.0,61.25,...,1263.75,46.305,4150.92400,2691.02975,152.27550,341.06900,19750.276,19559.139,4238.25,4400.50


In [20]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord.to_csv(filepath)


<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_7056\4209121318.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord.csv')


## Hourly for Macrozone Nord

In [21]:
import sys
print(sys.executable)

c:\Users\samueledelia\.conda\envs\myenv\python.exe


In [22]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_nord = qh[qh['MACROZONA'] == 'NORD']


# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour
h_nord = qh_nord.resample('H').sum()

# Clean the 'MACROZONA' column to retain only 'NORD'
h_nord['MACROZONA'] = h_nord['MACROZONA'].str[:4]

h_nord = h_nord.drop(columns = ['MACROZONA'])
# Display the first few rows of the resampled data
print(h_nord.head())

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_7056\1801842306.py:28: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  h_nord = qh_nord.resample('H').sum()


                     SBIL_MWH
ORAINI                       
2019-01-01 00:00:00  -111.900
2019-01-01 01:00:00  -112.451
2019-01-01 02:00:00   -88.849
2019-01-01 03:00:00   127.792
2019-01-01 04:00:00   162.699


In [23]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_ENERGY_BALANCE" 
power = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power = power.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
power = power[['ENERGY_BALANCE_GWh', 'ORAINI', 'ENERGY_SOURCE']]
# Convert ORAINI to datetime using the correct method
power['ORAINI'] = pd.to_datetime(power['ORAINI'], format='%Y%m%d%H%M')
power.set_index('ORAINI', inplace=True)
thermo = utils.filter_and_rename_energy_source(power, 'Thermal', 'THERMO_GWH')*1000
wind = utils.filter_and_rename_energy_source(power, 'Wind', 'WIND_GWH')*1000
solar = utils.filter_and_rename_energy_source(power, 'Photovoltaic', 'SOLAR_GWH')
hydro = utils.filter_and_rename_energy_source(power, 'Hydro', 'HYDRO_GWH')
geothermal = utils.filter_and_rename_energy_source(power, 'Geothermal', 'GEO_GWH')
forex = utils.filter_and_rename_energy_source(power, 'Net Foreign Exchange', 'FOREX_GWH')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [24]:
wind['WIND_MWH'] = wind['WIND_GWH'] * 1000
# Drop the original hourly GWh column
wind = wind.drop(columns=['WIND_GWH'])

solar['SOLAR_MWH'] = solar['SOLAR_GWH'] * 1000
# Drop the original hourly GWh column
solar = solar.drop(columns=['SOLAR_GWH'])

thermo['THERMO_MWH'] = thermo['THERMO_GWH'] * 1000
# Drop the original hourly GWh column
thermo = thermo.drop(columns=['THERMO_GWH'])

hydro['HYDRO_MWH'] = hydro['HYDRO_GWH'] * 1000
# Drop the original hourly GWh column
hydro = hydro.drop(columns=['HYDRO_GWH'])

geothermal['GEO_MWH'] = geothermal['GEO_GWH'] * 1000
# Drop the original hourly GWh column
geothermal = geothermal.drop(columns=['GEO_GWH'])

In [25]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_PHYSICAL_FOREIGN_FLOW" 
physical_forex = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
physical_forex = physical_forex.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
physical_forex = physical_forex[['PHYSICAL_FOREIGN_FLOW_MW', 'ORAINI', 'COUNTRY']]
# Convert ORAINI to datetime using the correct method
physical_forex['ORAINI'] = pd.to_datetime(physical_forex['ORAINI'], format='%Y%m%d%H%M')
physical_forex.set_index('ORAINI', inplace=True)

physical_forex

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,PHYSICAL_FOREIGN_FLOW_MW,COUNTRY
ORAINI,,
2019-01-01 00:00:00,156.0,Austria
2019-01-01 00:00:00,-142.0,Corsica
2019-01-01 00:00:00,-321.0,France
2019-01-01 00:00:00,-313.0,Greece
2019-01-01 00:00:00,-80.0,Malta
...,...,...
2024-11-25 23:00:00,1744.0,France
2024-11-25 23:00:00,494.0,Greece
2024-11-25 23:00:00,-89.0,Malta


In [26]:
def process_and_resample(df, country):
    country_data = utils.process_foreign_exchange(df, country)
    
    # Reset index if it's a MultiIndex
    country_data = country_data.reset_index()
    
    # Set 'ORAINI' as the index again for resampling
    country_data.set_index('ORAINI', inplace=True)
    
    # Resample and sum the values by hour
    country_data = country_data.resample('H').sum()
    
    return country_data

# Example usage for multiple countries:
austria_physical = process_and_resample(physical_forex, 'Austria')
corsica_physical = process_and_resample(physical_forex, 'Corsica')
france_physical = process_and_resample(physical_forex, 'France')
greece_physical = process_and_resample(physical_forex, 'Greece')
malta_physical = process_and_resample(physical_forex, 'Malta')
slovenia_physical = process_and_resample(physical_forex, 'Slovenia')
switzerland_physical = process_and_resample(physical_forex, 'Switzerland')
#montenegno_physical = process_and_resample(physical_forex, 'Montenegno')

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_7056\621014083.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_7056\621014083.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_7056\621014083.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_7056\621014083.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  country_data = country_data.resample('H').sum()
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_7056\621014083.py:11: FutureWarning: 'H' is deprecated 

In [27]:
austria_physical

,AUSTRIA_MWQH
ORAINI,
2019-01-01 00:00:00,156.00
2019-01-01 01:00:00,157.00
2019-01-01 02:00:00,147.00
2019-01-01 03:00:00,176.00
2019-01-01 04:00:00,165.00
...,...
2024-11-25 19:00:00,249.00
2024-11-25 20:00:00,258.00
2024-11-25 21:00:00,261.00


In [28]:
france_physical

,FRANCE_MWQH
ORAINI,
2019-01-01 00:00:00,-321.0
2019-01-01 01:00:00,-255.0
2019-01-01 02:00:00,-503.0
2019-01-01 03:00:00,-165.0
2019-01-01 04:00:00,-297.0
...,...
2024-11-25 19:00:00,2030.0
2024-11-25 20:00:00,2087.0
2024-11-25 21:00:00,2250.0


In [29]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,1843.786,0.0,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,1845.050,0.0,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,1845.647,0.0,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,983.111,0.0,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,978.775,0.0,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-26 19:00:00,42149.889,43347.924,3576.202,7388.522,23498.975,1055.693,2259.612,2548.437,0.0,None,...,NaN,0.0,3603.000,0.0,0.0,0.0,2786.0,40561.924,42809.889,2024-11-25 13:10:39.207
2024-11-26 20:00:00,39528.253,40688.253,3321.071,6995.963,22051.242,1036.215,2183.551,2485.317,0.0,None,...,NaN,0.0,3579.000,0.0,0.0,243.0,2786.0,37659.314,40188.253,2024-11-25 13:10:39.207
2024-11-26 21:00:00,36475.073,36975.073,3040.707,6432.743,20417.280,960.222,2018.808,2280.772,0.0,None,...,NaN,0.0,3535.838,624.8,0.0,390.0,2808.0,33777.045,36475.073,2024-11-25 13:10:39.207


In [30]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mgp_volumes_nord.rename(columns={"NORD_PURCHASES": "MGP_NORD_PURCHASES", "NORD_SALES": "MGP_NORD_SALES"}, inplace=True)
mgp_volumes_nord = mgp_volumes_nord[~mgp_volumes_nord.index.duplicated(keep='first')]


mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
mi1_volumes = mi1_volumes.drop(columns=['MARKET'])
mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mi1_volumes_nord.rename(columns={"NORD_PURCHASES": "MI1_NORD_PURCHASES", "NORD_SALES": "MI1_NORD_SALES"}, inplace=True)
mi1_volumes_nord = mi1_volumes_nord[~mi1_volumes_nord.index.duplicated(keep='first')]

In [31]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SCHEDULED_INTERNAL_EXCHANGE"
scheduled_internal = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
scheduled_internal = scheduled_internal.sort_values(by='ORAINI')

# Filter the DataFrame for the relevant columns and rows
scheduled_internal = scheduled_internal[['SCHEDULED_INTERNAL_EXCHANGE_MW', 'ORAINI', 'FLOW_FROM', 'FLOW_TO']]
# Convert ORAINI to datetime using the correct method
scheduled_internal['ORAINI'] = pd.to_datetime(scheduled_internal['ORAINI'], format='%Y%m%d%H%M')
scheduled_internal.set_index('ORAINI', inplace=True)
scheduled_internal = scheduled_internal[(scheduled_internal['FLOW_FROM'] == 'CNOR') & (scheduled_internal['FLOW_TO'] == 'NORD') ]
scheduled_internal_nord = scheduled_internal.drop(columns=['FLOW_FROM','FLOW_TO'])

scheduled_internal_nord

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,SCHEDULED_INTERNAL_EXCHANGE_MW
ORAINI,
2019-01-01 00:00:00,635.15
2019-01-01 01:00:00,940.59
2019-01-01 02:00:00,1372.39
2019-01-01 03:00:00,1617.07
2019-01-01 04:00:00,1921.50
...,...
2024-11-22 05:00:00,1040.29
2024-11-22 06:00:00,225.97
2024-11-22 12:00:00,375.76


In [32]:
# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_TOTAL_LOAD_QH" 
total_load = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
total_load = total_load.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
total_load = total_load[['ZONA', 'TOTAL_LOAD_MW', 'FORECAST_TOTAL_LOAD_MW', 'ORAINI']]
# Convert ORAINI to datetime using the correct method
total_load['ORAINI'] = pd.to_datetime(total_load['ORAINI'], format='%Y%m%d%H%M')
total_load.set_index('ORAINI', inplace=True)
subset_zonas = ['NORD']
mnord_load = utils.filter_load_by_zona(total_load, subset_zonas)
mnord_load = mnord_load.drop(columns=['ZONA'])
mnord_load = mnord_load.resample('H').sum()
mnord_load

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_7056\3996377423.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  mnord_load = mnord_load.resample('H').sum()


,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,47011.513,48015.081
2019-01-01 01:00:00,45175.351,46163.263
2019-01-01 02:00:00,42444.995,43497.895
2019-01-01 03:00:00,39593.045,41131.292
2019-01-01 04:00:00,38363.091,39863.454
...,...,...
2024-11-25 19:00:00,106633.652,106520.881
2024-11-25 20:00:00,99750.592,100468.322
2024-11-25 21:00:00,90521.518,91603.532


In [33]:
# Load the data
df_load_ensoe_path = r'C:\imbalance_forecast\data\df_load_entsoe.csv'
df_load_ensoe_path = pd.read_csv(df_load_ensoe_path, index_col=0, parse_dates=True)

# Access the relevant columns
mnord_load_entsoe = df_load_ensoe_path[['DAY_AHEAD_TOTAL_LOAD_MW_NORD', 'ACTUAL_TOTAL_LOAD_MW_NORD']]

# Check for duplicate timestamps
duplicates = mnord_load_entsoe.index.duplicated(keep=False)

if duplicates.any():
    # Handle duplicates: average values for duplicate timestamps
    mnord_load_entsoe = mnord_load_entsoe.groupby(mnord_load_entsoe.index).mean()


# Display the result
mnord_load_entsoe

,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,
2019-01-01 00:00:00,11110.0,11511.0
2019-01-01 01:00:00,10675.0,11039.0
2019-01-01 02:00:00,10080.0,10430.0
2019-01-01 03:00:00,9793.0,9811.0
2019-01-01 04:00:00,9485.0,9474.0
...,...,...
2024-11-04 08:00:00,21077.0,20756.0
2024-11-04 09:00:00,20997.0,20609.0
2024-11-04 10:00:00,NaN,NaN


In [34]:
lags = [-1, -2, -3, -24]

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(h_nord['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged = df_sbil_lagged.resample('H').sum()

df_sbil_lagged

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_7056\1260965639.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_sbil_lagged = df_sbil_lagged.resample('H').sum()


,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24
ORAINI,,,,
2019-01-01 00:00:00,0.000,0.000,0.000,0.000
2019-01-01 01:00:00,-111.900,0.000,0.000,0.000
2019-01-01 02:00:00,-112.451,-111.900,0.000,0.000
2019-01-01 03:00:00,-88.849,-112.451,-111.900,0.000
2019-01-01 04:00:00,127.792,-88.849,-112.451,0.000
...,...,...,...,...
2024-11-26 08:00:00,-347.169,-217.180,66.943,84.021
2024-11-26 09:00:00,-766.554,-347.169,-217.180,-132.764
2024-11-26 10:00:00,-646.470,-766.554,-347.169,-468.555


In [35]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [h_nord, df_sbil_lagged, wind, thermo, solar, hydro, geothermal, austria_physical, france_physical, slovenia_physical, switzerland_physical, scheduled_internal_nord, mgp_volumes_nord, mi1_volumes_nord, mnord_load, mnord_load_entsoe]
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

#df_nord_h = df_nord.drop(columns="MACROZONA")
df_nord_h = df_nord[df_nord.index <= '2024-03-11']
df_nord_h = df_nord_h[df_nord_h.index >= '2019-01-03']
df_nord_h

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24,WIND_MWH,THERMO_MWH,SOLAR_MWH,HYDRO_MWH,GEO_MWH,...,SWITZERLAND_MWQH,SCHEDULED_INTERNAL_EXCHANGE_MW,MGP_NORD_PURCHASES,MGP_NORD_SALES,MI1_NORD_PURCHASES,MI1_NORD_SALES,TOTAL_LOAD_MW,FORECAST_TOTAL_LOAD_MW,DAY_AHEAD_TOTAL_LOAD_MW_NORD,ACTUAL_TOTAL_LOAD_MW_NORD
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-03 00:00:00,42.456,-139.764,74.840,59.962,393.510,5610000.0,12970000.0,0.0,1630.0,670.0,...,2104.0,331.20,12884.917,8015.170,607.792,426.218,51751.436,54622.162,12778.0,12505.0
2019-01-03 01:00:00,-55.240,42.456,-139.764,74.840,470.408,5440000.0,12640000.0,0.0,1580.0,680.0,...,1595.0,887.58,12306.486,7970.890,519.879,538.758,49201.920,51573.941,11819.0,11674.0
2019-01-03 02:00:00,-46.830,-55.240,42.456,-139.764,477.902,4880000.0,12600000.0,0.0,1620.0,680.0,...,1552.0,1193.37,11961.762,7909.788,434.321,425.139,48006.133,49749.742,11309.0,11301.0
2019-01-03 03:00:00,-114.916,-46.830,-55.240,42.456,568.411,4920000.0,12670000.0,0.0,1560.0,680.0,...,1387.0,1379.92,11917.025,7838.137,351.280,415.050,47896.579,48849.077,11032.0,11241.0
2019-01-03 04:00:00,-65.532,-114.916,-46.830,-55.240,623.556,5000000.0,12720000.0,0.0,1620.0,680.0,...,1499.0,1478.67,12128.131,7881.073,367.500,399.960,48760.644,48803.013,11109.0,11454.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-10 20:00:00,-403.946,-589.230,-109.603,482.782,-296.668,4690000.0,9190000.0,0.0,8180.0,620.0,...,5055.0,NaN,16603.696,10764.119,609.102,1364.276,77733.373,77179.092,16953.0,17602.0
2024-03-10 21:00:00,-235.009,-403.946,-589.230,-109.603,-193.521,4180000.0,9130000.0,0.0,7080.0,620.0,...,4604.0,NaN,15750.399,10014.840,1065.549,1491.495,73127.189,73000.522,15950.0,16524.0
2024-03-10 22:00:00,-57.999,-235.009,-403.946,-589.230,-88.413,3890000.0,8870000.0,0.0,6010.0,620.0,...,4201.0,NaN,14706.304,9375.540,970.891,940.082,66840.142,66835.328,14745.0,15035.0


In [36]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord_h.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_7056\4230417481.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord_h.csv')
